## Geographically Weighted Correlation Coefficient

### Import relevant dependencies

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW

In [2]:
georgia_data = pd.read_csv(ps.examples.get_path('GData_utm.csv'))
georgia_shp = gp.read_file(ps.examples.get_path('G_utm.shp'))

#### Prepare the Georgia dataset inputs

In [3]:
g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB', 'PctBlack', 'PctRural']].values
u = georgia_data['X']
v = georgia_data['Y']


g_coords = list(zip(u,v))

#### Standardization Routine

I commented out the standardization routine here, because I will be doing it inside the _compute_betas_gwr()_ function of the `iwls.py`

#### Calibrate GWR model

In [4]:
gwr_selector = Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search()
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw).fit()

c:\Users\Lenovo\anaconda3\envs\standardweight\lib\site-packages\spglm\iwls.py:38: RuntimeWarning: invalid value encountered in true_divide
  x_stdz = (x - x.mean(axis=0)) / x.std(axis=0) # standa rdize the design matrix


ValueError: array must not contain infs or NaNs

In [ ]:
gwr_results.summary()

### Generate pearson's correlation coefficient

In [18]:
from scipy.stats import pearsonr

corr, u = pearsonr(g_X[:,1], g_y.flatten())

print(f'{corr:.2f}') # to 2 d.p

-0.11


 The cell below has my implementation of the _compute_betas_gwr()_ function from inside the conda environment `spglm/iwls.py` source code.  <br /> <br /> But when I do this and run the jupyter notebook cells I get the above error. 

In [ ]:
def _compute_betas_gwr(y, x, wi):
    """
    compute MLE coefficients using iwls routine

    Methods: p189, Iteratively (Re)weighted Least Squares (IWLS),
    Fotheringham, A. S., Brunsdon, C., & Charlton, M. (2002).
    Geographically weighted regression: the analysis of spatially varying relationships.
    """

    xw = (x * wi)  # weight before standardization -> result is the weighted design matrix

    x_stdz = (x - x.mean(axis=0)) / x.std(axis=0) # standardize the design matrix 

    xw_stdz = (xw - xw.mean(axis=0)) / xw.std(axis=0)  # standardize the weighted design matrix

    y = (y - y.mean(axis=0)) / y.std(axis=0)  # standardize the dependent variable --> Making sure x and y are standardized


    xT = xw_stdz.T
    xtx = np.dot(xT, x_stdz)
    xtx_inv_xt = linalg.solve(xtx, xT)
    betas = np.dot(xtx_inv_xt, y)
    return betas, xtx_inv_xt